In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, date, timedelta

pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 150)

* ** id**: user id
* ** date_account_created**: the date of account creation
* ** timestamp_first_active**: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
* ** date_first_booking**: date of first booking
* ** signup_flow**: the page a user came to signup up from
* ** language**: international language preference
* ** affiliate_channel**: what kind of paid marketing
* ** affiliate_provider**: where the marketing is e.g. google, craigslist, other
* ** first_affiliate_tracked**: whats the first marketing the user interacted with before the signing up
* ** country_destination**: this is the target variable you are to predict

-----------------------------------------------------
## Coutnry info data

In [2]:
countries_final_df = pd.read_pickle('./countries_age.pkl')
countries_final_df

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance,population_M,prc_young,avg_age
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00,23.923,55.99,42.53
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00,35.871,54.30,42.92
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61,82.562,47.42,44.46
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25,47.203,52.50,42.08
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06,64.983,48.96,46.21
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00,63.840,52.02,45.84
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40,61.145,46.53,40.13
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22,16.848,47.89,40.08
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45,10.609,51.61,42.16
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00,325.132,53.92,43.83


-----------------------------------------------------
## Train

In [3]:
_train_data_df = pd.read_csv('./assets/train_users_2.csv/train_users_2.csv')
print _train_data_df.shape
_train_data_df.info()

(213451, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


---------------------------------

In [4]:
train_data_df = _train_data_df.copy()
print train_data_df.shape
train_data_df.head()

(213451, 16)


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [ ]:
# train_data_df.country_destination.value_counts()
# train_data_df[train_data_df.country_destination == 'NDF'].count()#.info()

In [10]:
train_data_df.head(2)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,year_acc_created,month_acc_created,day_acc_created,dayofweek_acc_created,week_of_year_acc_created,year_1st_tmstmp,month_1st_tmstmp,day_1st_tmstmp,hour_1st_tmstmp,dayofyear_1st_tmstmp,week_of_year_1st_tmstmp,year_1st_booking,month_1st_booking,day_1st_booking,dayofweek_1st_booking,week_of_year_booking,daydf_accrtd_1bk,daydf_1tmstmp_accrtd,daydf_1tmstmp_1bk
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaT,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,2010,6,28,0,26,2009,3,19,4,3,12,NaN,NaN,NaN,NaN,NaN,NaT,-466 days +04:32:55,NaT
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaT,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,2011,5,25,2,21,2009,5,23,17,5,21,NaN,NaN,NaN,NaN,NaN,NaT,-732 days +17:48:09,NaT


In [11]:
columns = train_data_df.columns
columns[4:15]

Index([u'gender', u'age', u'signup_method', u'signup_flow', u'language',
       u'affiliate_channel', u'affiliate_provider', u'first_affiliate_tracked',
       u'signup_app', u'first_device_type', u'first_browser'],
      dtype='object')

In [12]:
for col in columns[4:15]:
    print col, 'total unique values:'
    print len(train_data_df[col].unique())
    print col, "unique values and count"
    print train_data_df[col].value_counts()
    print '----------------------------------------------------------'

gender total unique values:
4
gender unique values and count
-unknown-    95688
FEMALE       63041
MALE         54440
OTHER          282
Name: gender, dtype: int64
----------------------------------------------------------
age total unique values:
128
age unique values and count
30.0      6124
31.0      6016
29.0      5963
28.0      5939
32.0      5855
27.0      5738
33.0      5527
26.0      5044
34.0      5029
35.0      4860
25.0      4459
36.0      4083
37.0      3694
38.0      3384
24.0      3220
39.0      2998
40.0      2766
41.0      2538
23.0      2462
42.0      2243
45.0      2149
44.0      2137
43.0      2056
46.0      1875
22.0      1702
47.0      1646
48.0      1469
50.0      1387
51.0      1337
49.0      1331
          ... 
94.0        12
97.0        10
15.0         8
2.0          7
113.0        4
4.0          3
1932.0       3
1949.0       3
1931.0       3
1947.0       2
1936.0       2
1.0          2
1929.0       2
1928.0       2
111.0        2
1924.0       2
2008.0       1


In [31]:
train_data_df.first_affiliate_tracked.value_counts(dropna=False)

untracked        115297
linked            46287
omg               43982
tracked-other      6156
product            1556
marketing           139
local ops            34
Name: first_affiliate_tracked, dtype: int64

In [30]:
train_data_df.first_affiliate_tracked.fillna('untracked',inplace=True)

In [ ]:
# dict(zip(*np.unique(train_data_df['first_browser'],return_counts=True)))
# zip(*np.unique(d,return_counts=True))

In [ ]:
# train_data_df['affiliate_provider'].mask()

In [ ]:
# def change_to_other(cell, _list):
#     if cell in _list:
#         cell = 'Other'

### Fixing date columns

In [5]:
train_data_df.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
date_first_booking         124543
gender                          0
age                         87990
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6065
signup_app                      0
first_device_type               0
first_browser                   0
country_destination             0
dtype: int64

In [6]:
print type(train_data_df['date_account_created'][0])
print type(train_data_df['timestamp_first_active'][0])
print type(train_data_df['date_first_booking'][2])

<type 'str'>
<type 'numpy.int64'>
<type 'str'>


In [7]:
print train_data_df['date_account_created'][0]
print train_data_df['timestamp_first_active'][0]
print train_data_df['date_first_booking'][2]

2010-06-28
20090319043255
2010-08-02


In [8]:
train_data_df['date_account_created'] = pd.to_datetime(train_data_df['date_account_created'])
train_data_df['year_acc_created'] = train_data_df.date_account_created.dt.year
train_data_df['month_acc_created'] = train_data_df.date_account_created.dt.month
train_data_df['day_acc_created'] = train_data_df.date_account_created.dt.day
train_data_df['dayofweek_acc_created'] = train_data_df.date_account_created.dt.dayofweek
train_data_df['week_of_year_acc_created'] = train_data_df.date_account_created.dt.weekofyear

tmstmp = train_data_df['timestamp_first_active'].astype(str)
train_data_df['timestamp_first_active'] = tmstmp.apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S"))
train_data_df['year_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.year
train_data_df['month_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.month
train_data_df['day_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.day
train_data_df['hour_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.hour
train_data_df['dayofyear_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.dayofweek
train_data_df['week_of_year_1st_tmstmp'] = train_data_df.timestamp_first_active.dt.weekofyear

train_data_df['date_first_booking'] = pd.to_datetime(train_data_df['date_first_booking'])
train_data_df['year_1st_booking'] = train_data_df.date_first_booking.dt.year
train_data_df['month_1st_booking'] = train_data_df.date_first_booking.dt.month
train_data_df['day_1st_booking'] = train_data_df.date_first_booking.dt.day
train_data_df['dayofweek_1st_booking'] = train_data_df.date_first_booking.dt.dayofweek
train_data_df['week_of_year_booking'] = train_data_df.date_first_booking.dt.weekofyear

train_data_df['daydf_accrtd_1bk'] = (train_data_df.date_account_created - train_data_df.date_first_booking)#/86400.00
train_data_df['daydf_1tmstmp_accrtd'] = (train_data_df.timestamp_first_active - train_data_df.date_account_created)#/86400.00
train_data_df['daydf_1tmstmp_1bk'] = (train_data_df.timestamp_first_active - train_data_df.date_first_booking)#/86400.00

In [36]:
train_data_df.drop(['date_account_created', 'timestamp_first_active', 'date_first_booking'],axis=1,inplace=True)

In [9]:
train_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 35 columns):
id                          213451 non-null object
date_account_created        213451 non-null datetime64[ns]
timestamp_first_active      213451 non-null datetime64[ns]
date_first_booking          88908 non-null datetime64[ns]
gender                      213451 non-null object
age                         125461 non-null float64
signup_method               213451 non-null object
signup_flow                 213451 non-null int64
language                    213451 non-null object
affiliate_channel           213451 non-null object
affiliate_provider          213451 non-null object
first_affiliate_tracked     207386 non-null object
signup_app                  213451 non-null object
first_device_type           213451 non-null object
first_browser               213451 non-null object
country_destination         213451 non-null object
year_acc_created            213451 non-null int64


------------------

### Cleaning age column and creating age buckets

In [19]:
train_data_df.age.fillna(-1,inplace=True)
age_group = []
age_bckts = [21,25,28,32,37,45,55,65,75]
for cell in train_data_df.age:
#     print cell
    if cell< 0:
        aggp = '-unknown-'
#         age_group.append(aggp)
    elif (cell < 15) or (cell > 1900):
        if (cell< 15) or (cell>1998):#data is from 2014, minimum age for travel is assumed 15.
            aggp = 'Mistake'
#             print aggp
#             age_group.append(aggp)
#             break
        else:
            cell == 2014 - cell
    elif cell<=21:
            aggp = '15-21'
    elif cell>75:
            aggp = '75+'
    else:
        for i, agbkt in enumerate(age_bckts):
            if cell<=agbkt:
                aggp = str(age_bckts[i-1])+'-'+str(agbkt)
                break
    age_group.append(aggp)
train_data_df['age_bckts'] = age_group
# CC = [22,36,76,54,44,21,11,2,-1,-2,1923,2012,88,23]

In [20]:
train_data_df.age_bckts.unique()

array(['-unknown-', '37-45', '55-65', '45-55', '32-37', '28-32', '25-28',
       '15-21', '21-25', '65-75', 'Mistake', '75+'], dtype=object)

### Reducing number of features of some columns

#### Browser

In [22]:
brow_cnt = zip(train_data_df['first_browser'].value_counts().index, train_data_df['first_browser'].value_counts().values)
brow_other = [x[0] for x in brow_cnt if x[1]<150]
train_data_df.first_browser = train_data_df.first_browser.apply(lambda x: 'Other' if x in brow_other else x)

In [23]:
train_data_df.first_browser.unique()

array(['Chrome', 'IE', 'Firefox', 'Safari', '-unknown-', 'Mobile Safari',
       'Chrome Mobile', 'Other', 'Android Browser', 'AOL Explorer', 'Opera'], dtype=object)

#### Affiliate provider 

In [ ]:
# df_tt['n_null'] = np.array([sum(r == -1) for r in df_tt.values])
# df_tt

In [24]:
affprov_cnt = zip(train_data_df['affiliate_provider'].value_counts().index,
                  train_data_df['affiliate_provider'].value_counts().values)
affprov_other = [x[0] for x in affprov_cnt if x[1]<100]
train_data_df.affiliate_provider = train_data_df.affiliate_provider.apply(lambda x: 'Other' if x in affprov_other else x)

In [25]:
train_data_df.affiliate_provider.unique()

array(['direct', 'google', 'other', 'craigslist', 'facebook', 'vast',
       'bing', 'meetup', 'facebook-open-graph', 'email-marketing', 'yahoo',
       'padmapper', 'gsp', 'Other'], dtype=object)

In [32]:
airbnb_data = pd.merge(train_data_df,countries_final_df,on='country_destination')
print airbnb_data.shape
airbnb_data.info()

(78814, 45)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 78814 entries, 0 to 78813
Data columns (total 45 columns):
id                               78814 non-null object
date_account_created             78814 non-null datetime64[ns]
timestamp_first_active           78814 non-null datetime64[ns]
date_first_booking               78814 non-null datetime64[ns]
gender                           78814 non-null object
age                              78814 non-null float64
signup_method                    78814 non-null object
signup_flow                      78814 non-null int64
language                         78814 non-null object
affiliate_channel                78814 non-null object
affiliate_provider               78814 non-null object
first_affiliate_tracked          78814 non-null object
signup_app                       78814 non-null object
first_device_type                78814 non-null object
first_browser                    78814 non-null object
country_destination           

In [33]:
airbnb_data.to_csv('airbnb_data')

## Modelling

In [ ]:
# train_data_df = train_data_df.fillna(-1)  #Inputing this kind of missing value with -1 (missing values in train and test)
# train_data_df = train_data_df.replace('-unknown-', -1) #-unknown is another way of missing value, then = -1.

In [34]:
y = airbnb_data['country_destination']
airbnb_data.drop('country_destination',axis=1,inplace=True)
# users = train_data_df['id']
# train_data_df.drop('id',axis=1,inplace=True)

In [37]:
X = airbnb_data.copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78814 entries, 0 to 78813
Data columns (total 44 columns):
id                               78814 non-null object
date_account_created             78814 non-null datetime64[ns]
timestamp_first_active           78814 non-null datetime64[ns]
date_first_booking               78814 non-null datetime64[ns]
gender                           78814 non-null object
age                              78814 non-null float64
signup_method                    78814 non-null object
signup_flow                      78814 non-null int64
language                         78814 non-null object
affiliate_channel                78814 non-null object
affiliate_provider               78814 non-null object
first_affiliate_tracked          78814 non-null object
signup_app                       78814 non-null object
first_device_type                78814 non-null object
first_browser                    78814 non-null object
year_acc_created                 78814 non

In [42]:
# _X = X.dtypes.index
_x = [col for col in X.columns if X[col].dtype=='O']
_X = X[_x]

In [43]:
X = pd.get_dummies(_X,dummy_na=True)

MemoryError: 

In [ ]:
_X_.head()

In [ ]:
_X_.shape

In [ ]:
np.array(_X_.columns)

### Model

In [ ]:
from sklearn.datasets import load_svmlight_files
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier

In [ ]:
params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.25,
    'silent': 1.0,
    'n_estimators': 50
}

In [ ]:
bst = XGBClassifier(**params).fit(_X_, y)

In [ ]:
def get_xgb_imp(xgb, feat_names):
    from numpy import array
    imp_vals = xgb.booster().get_fscore()
    imp_dict = {feat_names[i]:float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))}
    total = array(imp_dict.values()).sum()
    return {k:v/total for k,v in imp_dict.items()}

In [ ]:
d = bst.booster().get_fscore()

In [ ]:
xgb_feat_importance = pd.DataFrame(d, index=index)
xgb_feat_importance

In [ ]:
d

In [ ]:
d